In [641]:
dataset_name = "webkb_parsed"

# Imports, load data , training functions and database creation

## Imports

In [642]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [643]:
import pandas as pd
import numpy as np
import matplotlib as plt
import os, errno
import networkx as nx
from networkx.algorithms import bipartite
import json
import yaml
import ast
from sklearn.metrics import classification_report
from sklearn.neighbors import NearestNeighbors
import pickle

import tensorflow as tf

In [644]:
import torch

os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

2.3.1+cu121
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [645]:
import torch_geometric
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.utils.convert import from_networkx
from torch_geometric.nn import SAGEConv, GATConv, Linear, to_hetero

In [646]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

## load data

In [647]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v2/processed_datasets/"+dataset_name)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [648]:
df = pd.read_pickle("/content/drive/MyDrive/Mestrado_Grafos/processed_datasets/"+dataset_name+"/"+dataset_name+".csv")

In [649]:
df.head()

,id,file_name,text,class,keyphrase2,keyphrase3,keyphrase23,text_embeddings,keyphrase2_embeddings,keyphrase3_embeddings,keyphrase23_embeddings,sentences_embeddings
0,0,http:^^www.cs.utexas.edu^users^dahlin^Classes^...,Content-Type: text/html Content-Length: 1978 L...,course,"[(web operating, 0.6921), (web related, 0.535)...","[(web operating systems, 0.7051), (purpose web...","[(web operating systems, 0.7051), (purpose web...","[-0.0927243, 0.0012164878, -0.05404062, -0.038...","[[-0.036058955, -0.01910096, -0.044949748, -0....","[[-0.045564666, -0.028437821, -0.032989483, -0...","[[-0.045564666, -0.028437821, -0.032989483, -0...","[[-0.010800143, 0.014000101, -0.044020705, -0...."
1,1,http:^^www.cs.rutgers.edu^~venugopa^112^112.html,CS 112: Data Structures CS 112: Data Struc...,course,"[(programming unix, 0.6077), (programming assi...","[(data structures cs, 0.6183), (programming as...","[(data structures cs, 0.6183), (programming un...","[-0.0030399715, 0.039208196, -0.009568916, -0....","[[-0.019407477, 0.030763928, -0.07262235, -0.0...","[[-0.036686115, 0.036067266, -0.03149169, 0.00...","[[-0.036686115, 0.036067266, -0.03149169, 0.00...","[[-0.0030399715, 0.039208196, -0.009568916, -0..."
2,2,http:^^www.eecs.umich.edu^courses^eecs661^eecs...,Set-Cookie: Apache=gs359343849035331353; path=...,course,"[(event systems, 0.5776), (systems eecs, 0.524...","[(discrete event systems, 0.6734), (event dyna...","[(discrete event systems, 0.6734), (event dyna...","[-0.07351037, -0.0034346038, -0.021328399, -0....","[[-0.022722067, 0.0026163666, 0.0029694512, -0...","[[-0.04119877, 0.008551315, -0.004188435, -0.0...","[[-0.04119877, 0.008551315, -0.004188435, -0.0...","[[-0.015701072, -0.001414385, -0.08331965, -0...."
3,3,http:^^www.cis.upenn.edu^~cse110e^cse110e.html,CSE 110 (Accelerated) Home Page CSE 110 (Acc...,course,"[(cse 110, 0.6049), (page cse, 0.5217), (intro...","[(cse 110 accelerated, 0.6174), (page cse 110,...","[(cse 110 accelerated, 0.6174), (cse 110, 0.60...","[-0.13189143, 0.035983704, 0.011182002, -0.010...","[[-0.12709178, 0.034261793, -0.008630871, 0.00...","[[-0.105777584, 0.0013491624, 0.007880913, 0.0...","[[-0.105777584, 0.0013491624, 0.007880913, 0.0...","[[-0.15982199, 0.02581229, -0.0061420402, -0.0..."
4,4,http:^^www.CS.Princeton.EDU^courses^archive^fa...,COS 320: Compiling Techniques. Fall 1995 ...,course,"[(computers grading, 0.5084), (programming ass...","[(cos 320 compiling, 0.5812), (computers gradi...","[(cos 320 compiling, 0.5812), (computers gradi...","[-0.10675096, -0.00079041265, -0.08070004, -0....","[[-0.030724177, 0.008653546, -0.05521948, -0.0...","[[-0.08890971, 0.05575665, -0.1463867, -0.0679...","[[-0.08890971, 0.05575665, -0.1463867, -0.0679...","[[-0.10500991, 0.087999925, -0.14258686, -0.06..."


In [650]:
classes = df["class"].unique()
print(classes)
class_number = len(classes)
print(class_number)

['course' 'department' 'faculty' 'other' 'project' 'staff' 'student']
7


--------------------------------------
# Hetrogenous Graph for Keyphrase = 2
--------------------------------------

In [651]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase2"

model_name = dataset_name+"_"+keyphrase

## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [652]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

# graph nodes (in embedded format)
document_nodes = []
context_nodes = []
# nodes' texts saved for identification in the coarsening method
document_nodes_text = []
context_nodes_text = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])
    document_nodes_text.append(df["text"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            context_nodes_text.append(df[keyphrase][i][j][0])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

In [653]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 8282
number of context nodes: 30369
number of shared contexts: 10924
number of direct edges (first dimension): 41293
number of direct edges (second dimension): 41293


In [654]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

##### Ajusting everything to Tensor Objects

In [655]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

In [656]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

In [657]:
# Show documents nodes
document_nodes

tensor([[-0.0927,  0.0012, -0.0540,  ...,  0.0492, -0.0104,  0.0809],
        [-0.0030,  0.0392, -0.0096,  ...,  0.0444, -0.0476,  0.0053],
        [-0.0735, -0.0034, -0.0213,  ...,  0.0422, -0.0553, -0.0198],
        ...,
        [-0.1017,  0.0156,  0.0452,  ...,  0.0244, -0.0924,  0.0201],
        [-0.0203, -0.0366, -0.0132,  ...,  0.0683, -0.0978,  0.0181],
        [ 0.0023, -0.0262,  0.0067,  ..., -0.0426, -0.1053,  0.0276]])

In [658]:
# Quantity of document nodes
len(document_nodes)

8282

In [659]:
# Show context nodes
context_nodes

tensor([[-0.0361, -0.0191, -0.0449,  ...,  0.0148,  0.0728,  0.0161],
        [-0.0383, -0.0169, -0.0790,  ...,  0.0370,  0.0679,  0.0069],
        [-0.0886, -0.0166, -0.0441,  ...,  0.0017,  0.0308,  0.0591],
        ...,
        [-0.0688,  0.0397,  0.0006,  ...,  0.0055,  0.0332, -0.0058],
        [-0.0183, -0.0137, -0.0364,  ...,  0.0738, -0.0228,  0.0223],
        [-0.0337, -0.0057, -0.0721,  ...,  0.0239,  0.0032, -0.0202]])

In [660]:
# Quantity of context nodes
len(context_nodes)

30369

In [661]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

2327488
2327488


In [662]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ...,  8281,  8281,  8281],
        [    0,     1,     2,  ..., 18257, 25278,  9640]])


#### Class Labels

In [663]:
# All different classes
print(df["class"].unique())

['course' 'department' 'faculty' 'other' 'project' 'staff' 'student']


In [664]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'course': 0, 'department': 1, 'faculty': 2, 'other': 3, 'project': 4, 'staff': 5, 'student': 6} 



<ipython-input-664-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


In [665]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([0, 0, 0,  ..., 6, 6, 6])

In [666]:
len(y)

8282

## Saving graph

In [667]:
 # cria diretorio para keypharse
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v3/processed_datasets/"+dataset_name+'/'+keyphrase)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [668]:
with open('/content/drive/MyDrive/mestrado_v3/processed_datasets/'+dataset_name+'/'+keyphrase+'/graph.pkl', 'wb') as f:
    aux = [document_nodes,context_nodes,edges,y,class_number,classes]
    pickle.dump(aux, f)

--------------------------------------
# Hetrogenous Graph for Keyphrase = 3
--------------------------------------

In [669]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase3"

model_name = dataset_name+"_"+keyphrase

## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [670]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

# graph nodes (in embedded format)
document_nodes = []
context_nodes = []
# nodes' texts saved for identification in the coarsening method
document_nodes_text = []
context_nodes_text = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])
    document_nodes_text.append(df["text"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            context_nodes_text.append(df[keyphrase][i][j][0])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

In [671]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 8282
number of context nodes: 36747
number of shared contexts: 4501
number of direct edges (first dimension): 41248
number of direct edges (second dimension): 41248


In [672]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

##### Ajusting everything to Tensor Objects

In [673]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

In [674]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

In [675]:
# Show documents nodes
document_nodes

tensor([[-0.0927,  0.0012, -0.0540,  ...,  0.0492, -0.0104,  0.0809],
        [-0.0030,  0.0392, -0.0096,  ...,  0.0444, -0.0476,  0.0053],
        [-0.0735, -0.0034, -0.0213,  ...,  0.0422, -0.0553, -0.0198],
        ...,
        [-0.1017,  0.0156,  0.0452,  ...,  0.0244, -0.0924,  0.0201],
        [-0.0203, -0.0366, -0.0132,  ...,  0.0683, -0.0978,  0.0181],
        [ 0.0023, -0.0262,  0.0067,  ..., -0.0426, -0.1053,  0.0276]])

In [676]:
# Quantity of document nodes
len(document_nodes)

8282

In [677]:
# Show context nodes
context_nodes

tensor([[-0.0456, -0.0284, -0.0330,  ...,  0.0154,  0.0630,  0.0540],
        [-0.0753,  0.0212, -0.0503,  ...,  0.0451,  0.0832,  0.0258],
        [-0.0521, -0.0103, -0.0148,  ...,  0.0641,  0.0069,  0.0194],
        ...,
        [ 0.0065, -0.0486, -0.0751,  ..., -0.0418,  0.0422, -0.0111],
        [-0.0202, -0.0181, -0.0205,  ...,  0.0454, -0.0772,  0.0521],
        [-0.0504, -0.0403, -0.0339,  ...,  0.0363,  0.0574, -0.0148]])

In [678]:
# Quantity of context nodes
len(context_nodes)

36747

In [679]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

2327443
2327443


In [680]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ...,  8281,  8281,  8281],
        [    0,     1,     2,  ..., 11807, 30834, 20437]])


#### Class Labels

In [681]:
# All different classes
print(df["class"].unique())

['course' 'department' 'faculty' 'other' 'project' 'staff' 'student']


In [682]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'course': 0, 'department': 1, 'faculty': 2, 'other': 3, 'project': 4, 'staff': 5, 'student': 6} 



<ipython-input-682-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


In [683]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([0, 0, 0,  ..., 6, 6, 6])

In [684]:
len(y)

8282

## Saving graph

In [685]:
 # cria dirrotio para keyphrase
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v3/processed_datasets/"+dataset_name+'/'+keyphrase)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [686]:
with open('/content/drive/MyDrive/mestrado_v3/processed_datasets/'+dataset_name+'/'+keyphrase+'/graph.pkl', 'wb') as f:
    aux = [document_nodes,context_nodes,edges,y,class_number,classes]
    pickle.dump(aux, f)

--------------------------------------
# Hetrogenous Graph for Keyphrase = 23
--------------------------------------

In [687]:
# Change here to change wich keypharse to use
keyphrase = "keyphrase23"

model_name = dataset_name+"_"+keyphrase

## Creating the Graph

### Defining Graph Nodes and Edges 👀

- `Nodes` - documents and contexts
- `Edges`
  - document <- has -> context
- `Labels` - documents classes


#### Nodes and Edges 👀

##### Defining Docmuent nodes, Context nodes and edges between them

In [688]:
all_contexts_list =[]
edges1,edges2 = [],[]
cont = 0
new_edge_cont = 0

# graph nodes (in embedded format)
document_nodes = []
context_nodes = []
# nodes' texts saved for identification in the coarsening method
document_nodes_text = []
context_nodes_text = []

edges_tuple = []

sentences = []
cont_sentences = 0
dit_sentences = {}

# Creating graph nodes and edges
for i in range(len(df)):
    # adding document nodes for every doc in df
    document_nodes.append(df["text_embeddings"][i])
    document_nodes_text.append(df["text"][i])

    # adding context node and edges:
    aux = df[keyphrase][i]
    for j,key in enumerate(aux):
        # testing if keyphrase was already detected in df
        if key[0] not in all_contexts_list: # if NOT
            # add a new context node for every new keyphrase in df
            context_nodes.append(df[keyphrase+"_embeddings"][i][j])
            context_nodes_text.append(df[keyphrase][i][j][0])
            # add a new edge between doc and new context
            edges1.append(df["id"][i])
            edges2.append(new_edge_cont)
            edges_tuple.append((df["id"][i],new_edge_cont))
            # update existing contexts list
            all_contexts_list.append(key[0])
            new_edge_cont += 1
        else: # if YES
            # we do NOT add context node and (already exists)
            # add a new edge between doc and old context
            edges1.append(df["id"][i])
            edges2.append(all_contexts_list.index(key[0]))
            edges_tuple.append((df["id"][i],all_contexts_list.index(key[0])))
            cont+=1

    # organize sentences, sentences_embeddings, and a dict with the corresponding document for each sentence
    aux = df['sentences_embeddings'][i]
    for j,key in enumerate(aux):
        sentences.append(key)
        dit_sentences[cont_sentences] = df["id"][i]
        cont_sentences += 1


document_nodes = np.array(document_nodes)
context_nodes = np.array(context_nodes)

In [689]:
print("number of document nodes:",len(document_nodes))
print("number of context nodes:",len(context_nodes))
print("number of shared contexts:",cont)
print("number of direct edges (first dimension):",len(edges1))
print("number of direct edges (second dimension):",len(edges2))

number of document nodes: 8282
number of context nodes: 36159
number of shared contexts: 5178
number of direct edges (first dimension): 41337
number of direct edges (second dimension): 41337


In [690]:
nbrs = NearestNeighbors(n_neighbors=15, metric='cosine').fit(context_nodes)

neighbors_list = nbrs.kneighbors(sentences, return_distance=False)

# cria aresta para cada vizinho encontrado
for i,neighbors in enumerate(neighbors_list):
        for n in neighbors:
            edges1.append(dit_sentences[i])
            edges2.append(n)
            edges_tuple.append((dit_sentences[i],n))

##### Ajusting everything to Tensor Objects

In [691]:
# transforms egdes to tensor
edges = np.array([edges1,edges2])
edges = torch.tensor(edges)

In [692]:
# transforms nodes to tensor
document_nodes = np.array(document_nodes)
document_nodes = torch.tensor(document_nodes)
context_nodes = np.array(context_nodes)
context_nodes = torch.tensor(context_nodes)

In [693]:
# Show documents nodes
document_nodes

tensor([[-0.0927,  0.0012, -0.0540,  ...,  0.0492, -0.0104,  0.0809],
        [-0.0030,  0.0392, -0.0096,  ...,  0.0444, -0.0476,  0.0053],
        [-0.0735, -0.0034, -0.0213,  ...,  0.0422, -0.0553, -0.0198],
        ...,
        [-0.1017,  0.0156,  0.0452,  ...,  0.0244, -0.0924,  0.0201],
        [-0.0203, -0.0366, -0.0132,  ...,  0.0683, -0.0978,  0.0181],
        [ 0.0023, -0.0262,  0.0067,  ..., -0.0426, -0.1053,  0.0276]])

In [694]:
# Quantity of document nodes
len(document_nodes)

8282

In [695]:
# Show context nodes
context_nodes

tensor([[-0.0456, -0.0284, -0.0330,  ...,  0.0154,  0.0630,  0.0540],
        [-0.0753,  0.0212, -0.0503,  ...,  0.0451,  0.0832,  0.0258],
        [-0.0361, -0.0191, -0.0449,  ...,  0.0148,  0.0728,  0.0161],
        ...,
        [-0.0688,  0.0397,  0.0006,  ...,  0.0055,  0.0332, -0.0058],
        [ 0.0065, -0.0486, -0.0751,  ..., -0.0418,  0.0422, -0.0111],
        [-0.0202, -0.0181, -0.0205,  ...,  0.0454, -0.0772,  0.0521]])

In [696]:
# Quantity of context nodes
len(context_nodes)

36159

In [697]:
# Quantity of edges
print(len(edges[0]))
print(len(edges[1]))

2327532
2327532


In [698]:
# showing edges
print(edges)

tensor([[    0,     0,     0,  ...,  8281,  8281,  8281],
        [    0,     1,     2,  ..., 11530, 20047,  4074]])


#### Class Labels

In [699]:
# All different classes
print(df["class"].unique())

['course' 'department' 'faculty' 'other' 'project' 'staff' 'student']


In [700]:
# Creating labels for classification
# the dictionary is a numeric representation for each possible "document" class

dit = {}
for i,classe in enumerate(df["class"].unique()):
  dit[classe] = i

print(dit,'\n')

labels = df[["class"]]
for i in range(len(df[["class"]])):
    labels["class"][i] = dit[labels["class"][i]]
labels.head()

{'course': 0, 'department': 1, 'faculty': 2, 'other': 3, 'project': 4, 'staff': 5, 'student': 6} 



<ipython-input-700-a018e46c97dd>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  labels["class"][i] = dit[labels["class"][i]]


,class
0,0
1,0
2,0
3,0
4,0


In [701]:
# tranfors class dataframe into tensor
y = labels["class"].tolist()
y = x_np = torch.tensor(y)
y

tensor([0, 0, 0,  ..., 6, 6, 6])

In [702]:
len(y)

8282

## Saving graph

In [703]:
 # cria diretorio para keyphrase
try:
    os.makedirs("/content/drive/MyDrive/mestrado_v3/processed_datasets/"+dataset_name+'/'+keyphrase)
except OSError as e:
    if e.errno != errno.EEXIST:
        raise

In [704]:
with open('/content/drive/MyDrive/mestrado_v3/processed_datasets/'+dataset_name+'/'+keyphrase+'/graph.pkl', 'wb') as f:
    aux = [document_nodes,context_nodes,edges,y,class_number,classes]
    pickle.dump(aux, f)